In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import time
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
"""# Load data
data = np.load("data/data_1.npy")
labels = np.load("data/labels_1.npy")

data = np.vstack((data,np.load("data/data_2.npy")))
labels = np.vstack((labels,np.load("data/labels_2.npy")))

data = np.vstack((data,np.load("data/data_3.npy")))
labels = np.vstack((labels,np.load("data/labels_3.npy")))

data = np.vstack((data,np.load("data/data_4.npy")))
labels = np.vstack((labels,np.load("data/labels_4.npy")))

data = np.vstack((data,np.load("data/data_5.npy")))
labels = np.vstack((labels,np.load("data/labels_5.npy")))"""

data = np.load("../data/images.npy")/255.
data = torch.FloatTensor(np.expand_dims(data,axis=1))
data_permuts = torch.randperm(data.shape[0])
data = data[data_permuts,:]
labels = torch.FloatTensor(np.load("../data/labels.npy"))
labels = labels[data_permuts,:]

print(f"data: {data.shape}, labels: {labels.shape}")

data: torch.Size([18000, 1, 150, 150]), labels: torch.Size([18000, 2])


<br/>
<h3>Labels transformation to cyclical</h3>

In [3]:
#we will use a dataframe to change the labels since it is more convenient.
labels_df = pd.DataFrame(labels,columns=['hour', 'minute'])
labels_df['h_cos'] = np.cos(2 * np.pi * labels_df["hour"] / labels_df["hour"].max())
labels_df['h_sin'] = np.sin(2 * np.pi * labels_df["hour"] / labels_df["hour"].max())
labels_df['m_cos'] = np.cos(2 * np.pi * labels_df["minute"] / labels_df["minute"].max())
labels_df['m_sin'] = np.sin(2 * np.pi * labels_df["minute"] / labels_df["minute"].max())

In [4]:
labels_df

,hour,minute,h_cos,h_sin,m_cos,m_sin
0,5.0,21.0,-0.959493,2.817324e-01,-0.617525,0.786551
1,1.0,53.0,0.841254,5.406408e-01,0.802712,-0.596367
2,4.0,3.0,-0.654861,7.557495e-01,0.949398,0.314077
3,10.0,1.0,0.841254,-5.406405e-01,0.994335,0.106293
4,11.0,37.0,1.000000,6.516827e-07,-0.697632,-0.716457
...,...,...,...,...,...,...
17995,2.0,5.0,0.415415,9.096320e-01,0.861554,0.507666
17996,10.0,34.0,0.841254,-5.406405e-01,-0.887352,-0.461093
17997,4.0,29.0,-0.654861,7.557495e-01,-0.998583,0.053222
17998,8.0,6.0,-0.142315,-9.898215e-01,0.802712,0.596367


In [5]:
labels_cycl = torch.FloatTensor(labels_df.iloc[:,2:].to_numpy())

In [6]:
labels_cycl

tensor([[-0.9595,  0.2817, -0.6175,  0.7866],
        [ 0.8413,  0.5406,  0.8027, -0.5964],
        [-0.6549,  0.7557,  0.9494,  0.3141],
        ...,
        [-0.6549,  0.7557, -0.9986,  0.0532],
        [-0.1423, -0.9898,  0.8027,  0.5964],
        [-0.9595,  0.2817, -0.5304,  0.8477]])

<br/>
<h3>Dataset creation</h3>

In [7]:
class ClockDataset(Dataset):
    """ Class used to create the pytorch DataLoaders.
    """
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [8]:
# Create main dataset
clock_dataset = ClockDataset(data, labels_cycl)
train_data, test_data, val_data = random_split(clock_dataset, [14000,3000,1000])

# Split dataset into train, test and validation sets
train_data = DataLoader(train_data, batch_size=64, shuffle=True)
test_data = DataLoader(test_data, batch_size=64, shuffle=True)
val_data = DataLoader(val_data, batch_size=64, shuffle=True)

In [9]:
for X, y in train_data:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 150, 150])
Shape of y: torch.Size([64, 4]) torch.float32


<br/>
<h3>Model creation</h3>

In [10]:
class NN_regression(nn.Module):
    """ Convolution model that returns one value as output.
    """
    def __init__(self, input_channels, h, w, n_outputs):
        super(NN_regression, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=5),
            nn.ReLU(),
            nn.BatchNorm2d(16)
        )
        self.hidden_layers = nn.Sequential(
            
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=2),
            nn.MaxPool2d(2),
            nn.Flatten()
        )


        self.output_layer = nn.Sequential(
            nn.Linear(156800, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, n_outputs)
            
        )


    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x

In [11]:
model = NN_regression(input_channels=1,h=150,w=150, n_outputs=4).to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train loss: {loss:>7f}")

In [13]:
def test(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss
    loss /= num_batches
    print(f"Test avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 50
start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, loss, optimizer)
    test(test_data, model, loss)
end_time = time.time()

Epoch 1
-------------------------------
